In [3]:
import pandas as pd
import json

In [4]:
with open("/home/ethereum/Downloads/ResearchData/chain/chain10k.json") as file:
    blockchain = json.load(file)

In [11]:
print(blockchain["chain"][1]["transactions"][1])

len(blockchain)

{'o_orderkey': '2', 'o_custkey': '547', 'o_totalprice': '65625.12', 'o_orderdate': '1996-12-01', 'timestamp': 1696930529.702839}


3

In [18]:
def check_freedelivery(total_spent, o_totalprice):
    threshold = 200000
    return (total_spent + float(o_totalprice) >= threshold)
        

def calc_totalspent(o_custkey, o_orderdate, blockchain):
    total_spent, flag = 0.0, 0
    
    for block_no in range(blockchain["length"]-1,0,-1):
        print("inside block", block_no)
        for tx_no in range(len(blockchain["chain"][block_no]["transactions"])-1, -1, -1):
            print("inside tx", tx_no)
            
            tx_date = blockchain["chain"][block_no]["transactions"][tx_no]["o_orderdate"]
            spent_time = abs(int(o_orderdate[:4]) - int(tx_date[:4]))*365 + abs(int(o_orderdate[5:7]) - int(tx_date[5:7]))*30 + abs(int(o_orderdate[8:]) - int(tx_date[8:]))
            print(spent_time)
            tx_price = float(blockchain["chain"][block_no]["transactions"][tx_no]["o_totalprice"])
            tx_custkey = blockchain["chain"][block_no]["transactions"][tx_no]["o_custkey"]
            tx_freedelivery = bool(blockchain["chain"][block_no]["transactions"][tx_no]["o_freedelivery"])
            
            if spent_time <= 365:
                if o_custkey == tx_custkey:
                    if tx_freedelivery == False:
                        total_spent += tx_price
                    else:
                        return total_spent
            else:
                return total_spent
    return total_spent

In [21]:
con = 0
for block in blockchain["chain"]:
    if con == 0:
        con = 1
        continue
    for tx in block["transactions"]:
        total_spent = calc_totalspent(tx["o_custkey"], tx["o_orderdate"], blockchain)
        tx["o_freedelivery"] = check_freedelivery(total_spent, tx["o_totalprice"])

inside block 13
inside tx 42
393


KeyError: 'o_freedelivery'